<a href="https://colab.research.google.com/github/sucrammal/vectari/blob/main/COSTAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai
from openai import AzureOpenAI
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import *

nltk.download('punkt')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [ ]:
from google.colab import userdata

# Setting up API
gpt_config = {
    'model': 'gpt-4-vectari-1106preview',
    'azure_endpoint': 'https://oai-vectari-eastus2.openai.azure.com/',
    'api_key': userdata.get('openAPIKey'),
    'api_version': '2023-12-01-preview'
}

# gpt_config = {
#     'model': 'gpt-35-turbo',
#     'azure_endpoint': 'https://oai-vectari-northcentral-us.openai.azure.com/',
#     'api_key': userdata.get('openAPIKey3.5'),
#     'api_version': '2024-02-15-preview'
# }

print(gpt_config['azure_endpoint'])

client = AzureOpenAI(
  azure_endpoint = gpt_config['azure_endpoint'],
  api_key = gpt_config['api_key'],
  api_version = gpt_config['api_version']
)

**Load data**

In [ ]:
# instantiate input file from scratch.
all_complaints = pd.read_csv("/content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/data/money_transfer_complaints.csv")

# extract complaints with transcripts
complaints_with_transcripts = all_complaints.loc[all_complaints["Consumer complaint narrative"].notna()]
complaints_with_transcripts.shape[0]

In [ ]:
# load LLM pre-cleaned transcripts from csv
input_file = input("Enter a file name that exists in the Data folder: ")
cleaned_dataset = pd.DataFrame()

with open(input_file, 'r') as file:
    cleaned_dataset = pd.read_csv(file)

**This function uses the LLM to clean incoming transcripts**

In [ ]:
cleaning_system = """
CONTEXT: You will receive a transcript of a phone call between a call center agent at a mortgage servicing company and a customer/borrower. You need to clean this transcript.

#############

OBJECTIVE: Your first job is to clean the incoming low-quality transcripts from the first dataset. These are calls about financial services and we have seen some text completely out of place.  For instance 'the first drug was fixed, and any drug after that was gonna be viral' should pretty clearly be
'the first rate was fixed, and any rate after that was going to be variable'.
Your job is to parse the text and using your natural language understanding and contextual awareness, please re-write the transcript and clean any grammar / things that do not make sense in financial use cases.
Additionally, remove any unwanted 'x's, 'xxxx's, any other similar phrases representing redaction, and simply any text or numbers that aren't words. Remove any dates, names, and otherwise sensitive information. After the response is generated, remove all full stops, commas, semicolons, colons, and quotation marks.

#############

# STYLE #
Financial complaint transcript

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: A PARAGRAPH OF TEXT #
"""

**Then, applying the cleaning function, ask the LLM to cluster the data**

In [ ]:
def clean_transcript(transcript):
  messages = [
    {
        "role":"system",
        "content": cleaning_system
    },
    {
        "role":"user",
        "content":f'"""{transcript}"""'
    }
  ]

  response = client.chat.completions.create(
      model=gpt_config['model'],
      messages = messages
  )

  LLM_cleaned_sample = response.choices[0].message.content
  return LLM_cleaned_sample

In [ ]:
cluster_system = """
CONTEXT: You will now be recieving a large text containing transcripts of phone calls between a call center agent at a mortgage servicing company and a customer/borrower. Individual transcripts
are separated by two newline characters, "\n\n", from one another. This large text represents a CLUSTER of transcripts that have been categorized based on content under some label, for example, "fraud or scam".

#############

OBJECTIVE:
Based on the cluster you have received,
1. CLUSTER_INFORMATION: Describe the cluster in terms of the dataset columns.
2. CLUSTER_CATEGORY_NAME: Interpret [CLUSTER_INFORMATION] to obtain a short name for the category of the transcripts in this cluster.

#############

# STYLE #
Business analytics report

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: MARKDOWN REPORT #
<From processing the CLUSTER you have received]>
– Complaint Group: [CLUSTER_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]

"""

In [ ]:
def cluster_complaints(df):
    clusters = df.groupby('Issue')
    cluster_info = {}

    for label, group in clusters:
        # Collect all transcripts in this cluster
        transcripts = group['Customer complaint narrative'].apply(clean_transcript).tolist()

        # Extract keywords and name cluster group.
        all_transcripts_joined = '\n\n'.join(transcripts)

        messages = [
          {
              "role":"system",
              "content": cluster_system
          },
          {
              "role":"user",
              "content":f'"""{all_transcripts_joined}"""'
          }
        ]

        response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

        cluster_report = response.choices[0].message.content


        cluster_info[label] = {
            'Report': cluster_report
        }

    return cluster_info

In [ ]:
classify_system = """
CONTEXT: You have previously created Complaint Groups of transcripts of complaint phone calls between a call center agent at a mortgage servicing company and a customer/borrower based
on the category of the complaints. Transcripts under each Complaint Group has been categorized based on content under some label, for example, "fraud or scam".

You will now be recieving two things: the first is a set of markdown reports representing the information you have gathered and organized for each CLUSTER, and are of the following format:
"– Complaint Group: [CLUSTER_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]"
Each markdown report for each Complaint Group will be separated by a new line character "\n".

The second piece of data you will receive is a transcript that has not been categorized into one of your CLUSTERs yet.

#############

OBJECTIVE: Based on your natural language understanding, and your understanding of the read and determine which Complaint Group, the incoming transcript belongs under.

#############

# STYLE #
Business analytics report

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: Single line of text containing ONLY one [CLUSTER_CATEGORY_NAME] of your choosing. #
[CLUSTER_CATEGORY_NAME]
"""

In [ ]:
def classify_transcripts(cluster_info, new_transcript):

  # unpack and format cluster_info, aka the reports
  markdown_reports = ""

  messages = [
          {
              "role":"system",
              "content": classify_system
          },
          {
              "role":"user",
              "content":f'"""{markdown_reports} \n {new_transcript}"""'
          }
        ]

  response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

  complaint_label = response.choices[0].message.content

  return complaint_label
